<a href="https://colab.research.google.com/github/VictorFBI/vkProfileTaskDataAnalysis/blob/main/vkProfileTaskDataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Перед запуском следующей строки необходимо подгрузить архив по соответствующему пути

In [1]:
! unzip /content/data_for_testing.zip

Archive:  /content/data_for_testing.zip
   creating: data_for_testing/
  inflating: data_for_testing/variant64.csv  
  inflating: data_for_testing/variant5.csv  
  inflating: data_for_testing/variant44.csv  
  inflating: data_for_testing/variant29.csv  
  inflating: data_for_testing/variant63.csv  
  inflating: data_for_testing/variant43.csv  
  inflating: data_for_testing/variant76.csv  
  inflating: data_for_testing/variant67.csv  
  inflating: data_for_testing/variant87.csv  
  inflating: data_for_testing/variant36.csv  
  inflating: data_for_testing/variant73.csv  
  inflating: data_for_testing/variant28.csv  
  inflating: data_for_testing/variant88.csv  
  inflating: data_for_testing/variant69.csv  
  inflating: data_for_testing/variant80.csv  
  inflating: data_for_testing/variant53.csv  
  inflating: data_for_testing/variant82.csv  
  inflating: data_for_testing/variant56.csv  
  inflating: data_for_testing/variant94.csv  
  inflating: data_for_testing/variant78.csv  
  inflatin

# Решение задачи

In [134]:
import pandas as pd


def filter_func(group):
    actions = group['action'].unique()
    if ('confirmation' in actions):
        return True
    return False


def get_best_days(df: pd.DataFrame) -> pd.DataFrame:
  sorted_df = df.sort_values(by=['timestamp'])
  sorted_df['timestamp'] = pd.to_datetime(sorted_df['timestamp'])
  sorted_df['month'] = sorted_df['timestamp'].dt.month
  first_month = sorted_df.iloc[0]['month']
  last_month = sorted_df.iloc[-1]['month']
  indexes = []

  for i in range(0, sorted_df.shape[0]):
    if sorted_df.iloc[i]['month'] == first_month:
      indexes.append(sorted_df.index[i])
    else:
      break


  for i in range(sorted_df.shape[0] - 1, -1, -1):
    if sorted_df.iloc[i]['month'] == last_month:
      indexes.append(sorted_df.index[i])
    else:
      break

  deleted_df = sorted_df.drop(indexes, axis=0) # удаляем первые и последние месяца, так как в них могут быть неполные данные
  deleted_df['timestamp'] = deleted_df['timestamp'].dt.to_period('D')
  deleted_df = deleted_df.drop(['month'], axis=1)

  grouped_df = deleted_df.groupby(['timestamp', 'userid']) # группируем сначала по дню, потом по пользователю
  filtered_groups = grouped_df.filter(filter_func) # фильтруем только те группы, в которых было совершено подтверждение покупки

  final_df = filtered_groups[filtered_groups['action'] == 'confirmation'] # оставляем в записях только те, для которых action == confirmation
  final_df = final_df.groupby('timestamp')['value'].sum() # находим сумму выручки в каждый из дней

  day2value_df = pd.DataFrame(final_df).sort_values(by=['timestamp'], ascending=[True]) # сортируем по отметке времени
  day2value_df['timestamp'] = day2value_df.index
  month2value_df = day2value_df.resample('M').max() # по месяцу храним максимальную сумму, вырученную в один из дней этого месяца
  month2value_df['month'] = month2value_df['timestamp'].dt.strftime('%Y-%m') # записываем год и месяц
  day2value_df['month'] = day2value_df['timestamp'].dt.strftime('%Y-%m') # записываем год и месяц
  month2value_df.drop('timestamp', axis=1, inplace=True)

  # Сбрасываем индексы (которые до этого были timestamp)
  month2value_df = month2value_df.reset_index(drop=True)
  day2value_df = day2value_df.reset_index(drop=True)

  ans = pd.merge(day2value_df, month2value_df, on=['value', 'month']) # Выполняем join по месяцу и максимальной выручке
  ans.drop('month', axis=1, inplace=True)
  ans = ans[['timestamp', 'value']]
  return ans

# Тестирование

In [135]:
import numpy as np


np.random.seed(42) # фиксируем seed для воспроизводимости результатов
test = 1
for i in np.random.randint(1, 101, size=10):
  df = pd.read_csv(f'data_for_testing/variant{i}.csv', sep='\t')
  print("###########################################")
  print(f"Test №{test}, dataframe №{i}")
  test += 1
  print(get_best_days(df).head(10))

###########################################
Test №1, dataframe №52
    timestamp   value
0  2021-04-22  235637
1  2021-05-29  207717
2  2021-06-03  294750
3  2021-07-31  255912
4  2021-08-25  279787
5  2021-09-23  267209
6  2021-10-06  186793
7  2021-11-21  243440
8  2021-12-25  185144
9  2022-01-12  180066
###########################################
Test №2, dataframe №93
    timestamp   value
0  2021-04-05  230161
1  2021-05-10  297831
2  2021-06-06  166088
3  2021-07-11  278290
4  2021-08-15  241070
5  2021-09-15  223000
6  2021-10-14  199477
7  2021-11-21  167523
8  2021-12-27  423746
9  2022-01-12  249417
###########################################
Test №3, dataframe №15
    timestamp   value
0  2021-05-29  297981
1  2021-06-15  169304
2  2021-07-03  288436
3  2021-08-14  186108
4  2021-09-21  233459
5  2021-10-22  246209
6  2021-11-29  243356
7  2021-12-24  258270
8  2022-01-05  198811
9  2022-02-03  320081
###########################################
Test №4, dataframe №72
    ti